In [12]:
import torch
import numpy as np
import pandas as pd
from sklearn import datasets
import torch.nn.functional as F
import torch.nn as nn 
from torch import optim
from sklearn.model_selection import train_test_split
import time
import copy

In [2]:
breast_cancer=datasets.load_breast_cancer()
X=breast_cancer.data
Y=breast_cancer.target
print(X.shape,Y.shape)

(569, 30) (569,)


In [3]:
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
X = mm_scaler.fit_transform(X)
print(X)

[[0.52103744 0.0226581  0.54598853 ... 0.91202749 0.59846245 0.41886396]
 [0.64314449 0.27257355 0.61578329 ... 0.63917526 0.23358959 0.22287813]
 [0.60149557 0.3902604  0.59574321 ... 0.83505155 0.40370589 0.21343303]
 ...
 [0.45525108 0.62123774 0.44578813 ... 0.48728522 0.12872068 0.1519087 ]
 [0.64456434 0.66351031 0.66553797 ... 0.91065292 0.49714173 0.45231536]
 [0.03686876 0.50152181 0.02853984 ... 0.         0.25744136 0.10068215]]


In [4]:
#splitting data into  80% train and 20% test
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,stratify=Y,random_state=4,test_size=0.20)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(455, 30) (114, 30) (455,) (114,)


In [26]:
#covert all data into tensors
X_train,X_test,Y_train,Y_test=map(torch.tensor ,(X_train,X_test,Y_train,Y_test))
print(X_train.dtype, X_test.dtype, Y_train.dtype, Y_test.dtype)
#print(Y_train)

torch.float32 torch.float32 torch.int64 torch.int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [6]:
#convert all tensors to float to avoid in training
X_train=X_train.float()
X_test=X_test.float()
Y_train=Y_train.long()
Y_test=Y_test.long()

In [7]:
#constructing class to built neural network
class Classfication(nn.Module):
  def __init__(self):
     super().__init__()
     torch.manual_seed(0)
     self.net=nn.Sequential(
         nn.Linear(30,50),
         nn.ReLU(),
         nn.Linear(50,100),
         nn.ReLU(),
         nn.Linear(100,50),
         nn.ReLU(),
         nn.Linear(50,2),
         nn.Softmax()
     )
  def forward(self,x):
    return self.net(x)

In [ ]:
 def accuracy(X,y):
    y_hat=model(X)
    y_np=y_hat.detach().cpu().numpy()
    y_model=np.argmax(y_np,axis=1)
    y_test=y.detach().cpu().numpy()
    count=0
    for y_pred,y_true in zip(y_model,y_test):
      if(y_pred==y_true):
          count=count + 1
    accuracy=(count/y_model.shape[0])*100
    #print("accuracy:",accuracy)
    return accuracy

In [17]:
def fit_gpu(X_train,X_test,Y_test,Y_train,model,opt,loss_fn,epochs=100):
        #callback=StopatLossValue()
        time0=time.time()
        trainloss=[]
        valloss=[]
        trainacclist=[]
        valacclist=[]
        min_valloss=10000000.0000000
        min_trainloss=10000000.0000000
        max_valacc=0.0
        max_trainacc=0.0
        for epoch in range(epochs):
            runningtrain_loss=0
            runningval_loss=0
            vallen=114
            trainlen=455
            #init_fn()
            inputs, labels = X_train,Y_train
            inputs=inputs.to(device)
            labels=labels.to(device)

                #forward pass
            loss = loss_fn(model(inputs), labels)

                #backward and optimize
            opt.zero_grad()
            loss.backward()
            opt.step()
            trainacc=accuracy(inputs,labels)   
                #loss_arr.append(loss.item())
            runningtrain_loss += loss.item()
            #init_fn()
            inputs, labels = X_test,Y_test
            inputs=inputs.to(device)
            labels=labels.to(device)

                #forward pass
            loss = loss_fn(model(inputs), labels)
                #valloss_arr.append(loss.item())
            valacc=accuracy(inputs,labels)
            runningval_loss += loss.item()
            avgval_loss=runningval_loss/vallen
            avgtrain_loss=runningtrain_loss/trainlen
            if (avgval_loss<min_valloss):
                min_valloss=avgval_loss    
                vallossmodel=copy.deepcopy(model.state_dict())
            if (avgtrain_loss<min_trainloss):
                min_trainloss=avgtrain_loss
                trainlossmodel=copy.deepcopy(model.state_dict())
            
            
            trainacclist.append(trainacc)
            valacclist.append(valacc)
            if (trainacc>max_trainacc):
                max_trainacc=trainacc
                trainaccmodel=copy.deepcopy(model.state_dict())
            if (valacc>max_valacc):
                max_trainacc=valacc
                valaccmodel=copy.deepcopy(model.state_dict())      
            print("Minimum training Loss :",min_trainloss,"| Minimum validation Loss :",min_valloss) 
            print("Epoch {} - Training loss: {}".format(epoch,avgtrain_loss ),"| validation loss: {}".format(avgval_loss))    
            print('Epoch: %d/%d, Train acc: %0.2f' % (epoch, epochs-1,trainacc ),'| val acc: %0.2f' % (valacc))
            print('------------------------------------------------------------------------------------------')
            trainloss.append(avgtrain_loss)       
            valloss.append(avgval_loss)
            #vis.line(Y=avgtrain_loss, opts=dict(showlegend=True))
        print("Training Time(in minutes) =",(time.time()-time0)/60)    
        return vallossmodel,trainlossmodel,valaccmodel,trainaccmodel,trainloss,valloss,trainacclist,valacclist

In [18]:
device=torch.device("cuda")

In [25]:
model=Classfication()
model=model.to(device)
loss_fn=F.cross_entropy
opt=optim.Adam(model.parameters(),lr=0.01)
vallossmodel,trainlossmodel,valaccmodel,trainaccmodel,trainloss,valloss,trainacclist,valacclist=fit_gpu(X_train,X_test,Y_test,Y_train,model,opt,loss_fn)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Minimum training Loss : 0.001517535041976761 | Minimum validation Loss : 0.005914659353724697
Epoch 0 - Training loss: 0.001517535041976761 | validation loss: 0.005914659353724697
Epoch: 0/99, Train acc: 62.64 | val acc: 63.16
------------------------------------------------------------------------------------------
Minimum training Loss : 0.0014835108767498981 | Minimum validation Loss : 0.005748351937846134
Epoch 1 - Training loss: 0.0014835108767498981 | validation loss: 0.005748351937846134
Epoch: 1/99, Train acc: 62.64 | val acc: 63.16
------------------------------------------------------------------------------------------
Minimum training Loss : 0.0014440784087547888 | Minimum validation Loss : 0.005505496995490894
Epoch 2 - Training loss: 0.0014440784087547888 | validation loss: 0.005505496995490894
Epoch: 2/99, Train acc: 63.08 | val acc: 63.16
------------------------------------------------------------------------------------------
Minimum training Loss : 0.0013868060740795